# import

In [1]:
from skimage.feature import daisy
import numpy as np
import joblib
# To read image file and save image feature descriptions
import os
import time
# import glob
import pickle as pk
from config import *
import operator
from functools import reduce
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

# read data

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def getData(filePath):
    TrainData = []
    flag_train=0
    flag_test=0


    for childDir in os.listdir(filePath):
        if 'data_batch' in childDir:
            f = os.path.join(filePath, childDir)
            data = unpickle(f)
            # train = np.reshape(data[str.encode('data')], (10000, 3, 32 * 32))
            # If your python version do not support to use this way to transport str to bytes.
            # Think another way and you can.
            train = np.reshape(data[b'data'], (10000, 3, 32 * 32))
            if(flag_train==0):
                train_label=(np.reshape(data[b'labels'], (10000, 1)))
                flag_train=1
            else:
                train_label=np.concatenate((train_label,np.reshape(data[b'labels'], (10000, 1))))

            datalebels = zip(train)
            TrainData.extend(datalebels)


        cnt=0
        if childDir == "test_batch":
            f = os.path.join(filePath, childDir)
            data = unpickle(f)
            test = np.reshape(data[b'data'], (10000, 3, 32 * 32))
            if (flag_test == 0):
                test_label = (np.reshape(data[b'labels'], (10000, 1)))
                flag_test = 1
            else:
                test_label = np.concatenate((test_label, np.reshape(data[b'labels'], (10000, 1))))
            TestData = zip(test)

    train_label=train_label.ravel()
    test_label=test_label.ravel()
    return TrainData, TestData,train_label,test_label

# get Daisy Feature

In [7]:
def getFeat(TrainData, TestData):

    #这两个数组，注意，列的个数和HOG参数选择有关，不知道怎么自适应所以不调了
    train_feature=np.zeros(shape=(50000,200))
    test_feature=np.zeros(shape=(10000,200))
    cnt=0
    for data in TestData:
        image = np.reshape(data[0].T, (32, 32, 3))
        gray = rgb2gray(image)/255.0
        fd = daisy(gray)

        test_feature[cnt]=fd
        cnt=cnt+1

    print("Test features are extracted and saved.")
    cnt=0
    for data in TrainData:
        image = np.reshape(data[0].T, (32, 32, 3))
        gray = rgb2gray(image)/255.0
        fd = daisy(gray)

        train_feature[cnt] = fd
        cnt = cnt + 1

    print("Train features are extracted and saved.")

    return train_feature,test_feature


def rgb2gray(im):
    gray = im[:, :, 0]*0.2989+im[:, :, 1]*0.5870+im[:, :, 2]*0.1140
    return gray

# main

In [8]:
filePath = r'C:\Users\包广垠\PythonWorkPlace\ML-task2\cifar-10-batches-py'
TrainData, TestData, y_train, y_test = getData(filePath)
x_train_hog,x_test_hog = getFeat(TrainData, TestData)
print("Features are extracted")

Test features are extracted and saved.
Train features are extracted and saved.
Features are extracted


# model

In [9]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import tree
from sklearn import naive_bayes

import xgboost
from sklearn import ensemble

## Prediction using Bernoulli Naive Bayes

In [10]:
## Training
bbn = naive_bayes.BernoulliNB()
bbn.fit(x_train_hog, y_train)

BernoulliNB()

In [11]:
y_pred_bbn = bbn.predict(x_test_hog)
bernoulli_naive_bayes_score = accuracy_score(y_test, y_pred_bbn)
bernoulli_naive_bayes_score

0.1

## Prediction using Gaussian Naive Bayes

In [12]:
## Training
gbn = naive_bayes.GaussianNB()
gbn.fit(x_train_hog, y_train)

GaussianNB()

In [13]:
## Predicting
y_pred_gbn = gbn.predict(x_test_hog)
gaussian_naive_bayes_score = accuracy_score(y_test, y_pred_gbn)
gaussian_naive_bayes_score

0.3839

## Prediction using Random Forest

In [14]:
## Training
rf = RandomForestClassifier()
rf.fit(x_train_hog, y_train)

RandomForestClassifier()

In [15]:
## Predicting
y_pred_rf = rf.predict(x_test_hog)
random_forest_score = accuracy_score(y_test, y_pred_rf)
random_forest_score

0.5228

## Prediction using KNN

In [16]:
## Training 
knn = KNeighborsClassifier()
knn.fit(x_train_hog, y_train)

KNeighborsClassifier()

In [17]:
## Predicting
y_pred_knn = knn.predict(x_test_hog)
knn_score = accuracy_score(y_test, y_pred_knn)
knn_score

0.4859

## Prediction using Logistic Regression

In [18]:
## Training 
lr = LogisticRegression()
lr.fit(x_train_hog, y_train)

LogisticRegression()

In [19]:
## Predicting
y_pred_lr = lr.predict(x_test_hog)
logistic_regression_score = accuracy_score(y_test, y_pred_lr)
logistic_regression_score

0.3604

## Prediction using Linear SVM

In [20]:
## Training
lsvc = svm.LinearSVC()
lsvc.fit(x_train_hog, y_train)

LinearSVC()

In [21]:
## Predicting 
y_pred_lsvm = lsvc.predict(x_test_hog)
linear_svc_score = accuracy_score(y_test, y_pred_lsvm)
linear_svc_score

0.4117

## Prediction using SVM

In [22]:
## Training
svc = svm.SVC()
svc.fit(x_train_hog, y_train)

SVC()

In [23]:
## Predicting 
y_pred_svm = svc.predict(x_test_hog)
svc_score = accuracy_score(y_test, y_pred_svm)
svc_score

0.6427

## prediction using Decision Tree

In [24]:
## Training
dt = tree.DecisionTreeClassifier()
dt.fit(x_train_hog, y_train)

DecisionTreeClassifier()

In [25]:
## Predicting
y_pred_dt = dt.predict(x_test_hog)
decision_tree_score = accuracy_score(y_test, y_pred_dt)
decision_tree_score

0.3145

## prediction using XGBoost

In [26]:
## Training
xgb = xgboost.XGBClassifier()
xgb.fit(x_train_hog, y_train)

C:\Users\包广垠\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:41:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [27]:
## Predicting
y_pred_xgb = xgb.predict(x_test_hog)
XGBoost_score = accuracy_score(y_test, y_pred_xgb)
XGBoost_score

0.6063

## prediction using AdaBoost

In [28]:
## Training
adb = ensemble.AdaBoostClassifier()
adb.fit(x_train_hog, y_train)

AdaBoostClassifier()

In [29]:
## Predicting
y_pred_adb = adb.predict(x_test_hog)
AdaBoost_score = accuracy_score(y_test, y_pred_adb)
AdaBoost_score

0.3648

## Comparison between various Classifier

In [30]:
print("Bernoulli Naive Bayes : ", bernoulli_naive_bayes_score)
print("Gaussian Naive Bayes : ", gaussian_naive_bayes_score)
print("Random Forest Classifier : ", random_forest_score)
print("K Nearest Neighbors : ", knn_score)
print("Logistic Regression : ", logistic_regression_score)
print("Linear Support Vector Classifier : ", linear_svc_score)
print("Support Vector Classifier : ", svc_score)
print("Decision Tree Classifier : ", decision_tree_score)
print("XGBoost : ", XGBoost_score)
print("AdaBoost : ", AdaBoost_score)

Bernoulli Naive Bayes :  0.1
Gaussian Naive Bayes :  0.3839
Random Forest Classifier :  0.5228
K Nearest Neighbors :  0.4859
Logistic Regression :  0.3604
Linear Support Vector Classifier :  0.4117
Support Vector Classifier :  0.6427
Decision Tree Classifier :  0.3145
XGBoost :  0.6063
AdaBoost :  0.3648
